# Tarea PySpark

### Objetivo:
 Analizar la eficiencia de los jugadores en términos generales y por posición, así como determinar la contribución al equipo por jugador tomando en cuenta los datos obtenidos

Usarás la base de datos del archivo 'fusbol.csv' para obtener tus datos. Checa la estructura del archivo para ver si es necesario limpiar la informacion, ver su estructura y así sea más fácil completar la tarea. Besos.

### Ejercicio 1:
Carga la base de datos en un DataFrame de Pyspark (con 2 nucleos). Valida los rangos de los valores donde sea aplicable, así como su corrección (en caso de ser necesaria). 
Después, utilizando las variables más relevantes como 'Ast/90', 'PassCmp%', etc., concluye qué ligas tienen los mejores jugadores por posición. Es decir, si los mejores jugadores defensas son de la liga francesa, inglesa, etc., por ejemplo.
Como cada persona tiene una definición de "mejor", utiliza las siguientes metricas por posición:
- Delanteros: npG+A/90 y npxG+xA/90
- Medios: KeyPass/90 y PassCmp%
- Defensas: PressSucc% y Interceptions/90

Con los resultados obtenidos, grafica por posición para que tu conclusión tenga un respaldo visual también.

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, avg, percent_rank
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
spark = SparkSession.builder.appName('tareaPySpark')\
    .config('spark.master', 'local[2]')\
    .config('spark.executor.memory', '1g')\
    .config("spark.sql.shuffle.partitions", 10)\
    .config('spark.driver.memory','1g')\
    .getOrCreate()

In [ ]:
df_futbol = spark.read.csv('fusbol.csv', header=True, inferSchema=True)
df_futbol.show()

In [ ]:
#Observamos la estructura del df para checar que los dateos sean coherentes
df_futbol.printSchema()

In [ ]:
#Tiramos nulls
df_futbol= df_futbol.dropna()
df_futbol.count()

In [ ]:
#Validamos los porcentajes que vamos a usar (PassCmp% y PressSucc%)
#Si está por arriba de 100 lo toopamos en 100, si está por debajo de 0 lo ponemos en 0
df_futbol = df_futbol.withColumn('PassCmp%', when(col('PassCmp%') > 100, 100).otherwise(col('PassCmp%')))
df_futbol = df_futbol.withColumn('PassCmp%', when(col('PassCmp%') < 0, 0).otherwise(col('PassCmp%')))
df_futbol = df_futbol.withColumn('PressSucc%', when(col('PressSucc%') > 100, 100).otherwise(col('PressSucc%')))
df_futbol = df_futbol.withColumn('PressSucc%', when(col('PressSucc%') < 0, 0).otherwise(col('PressSucc%')))

In [ ]:
#Incluimos todos los que sean delanteros, aunque también jueguen de medios
delanteros=df_futbol.filter(col('pos').like("%FW%"))

In [ ]:
#Agrupamos por liga
delanteros_stats=delanteros.groupBy('Comp').agg(avg('npG+A/90').alias('avg_npG+A/90'), avg('npxG+xA/90').alias('avg_npxG+xA/90'))
delanteros_stats_pd=delanteros_stats.toPandas()
delanteros_stats.show()

Parece ser que los mejores delanteros son los de la Serie A, seguidos de cerca por los de la Bundesliga

In [ ]:
# Configuración de la figura
plt.figure(figsize=(10, 8))

# Crear un scatter plot
for comp in delanteros_stats_pd['Comp'].unique():
    subset = delanteros_stats_pd[delanteros_stats_pd['Comp'] == comp]
    plt.scatter(subset['avg_npG+A/90'], subset['avg_npxG+xA/90'], label=comp, s=50)  # s controla el tamaño del punto

# Añadir leyenda
plt.legend(title='Competition')

# Añadir títulos y etiquetas
plt.title('Comparison of Average Performance Metrics by Competition')
plt.xlabel('Average Goals and Assists per 90')
plt.ylabel('Average Expected Goals and Assists per 90')

# Mostrar la gráfica
plt.show()

In [ ]:
medios=df_futbol.filter(col('pos').like("%MF%"))

In [ ]:
medios_stats=medios.groupBy('Comp').agg(avg('KeyPass/90').alias('avg_KeyPass/90'), avg('PassCmp%').alias('avg_PassCmp%'))
medios_stats_pd=medios_stats.toPandas()
medios_stats.show()

En este caso es un poco más difícil ver qué liga es superior. Depende cuál de las dos estadísticas valoremos más. Sin embargo, parece que las mejores son la Serie A, mientras los peores son los de La Liga.

In [ ]:
# Configuración de la figura
plt.figure(figsize=(10, 8))

# Crear un scatter plot
for comp in medios_stats_pd['Comp'].unique():
    subset = medios_stats_pd[medios_stats_pd['Comp'] == comp]
    plt.scatter(subset['avg_KeyPass/90'], subset['avg_PassCmp%'], label=comp, s=50)  # s controla el tamaño del punto

# Añadir leyenda
plt.legend(title='Competition')

# Añadir títulos y etiquetas
plt.title('Comparison of Average Midfield Performance Metrics by Competition')
plt.xlabel('Average Key Passes per 90')
plt.ylabel('Average Pass Completion %')

# Mostrar la gráfica
plt.show()

In [ ]:
defensas=df_futbol.filter(col('pos').like("%DF%"))

In [ ]:
defensas_stats=defensas.groupBy('Comp').agg(avg('PressSucc%').alias('avg_PressSucc%'), avg('Interceptions/90').alias('avg_Interceptions/90'))
defensas_stats_pd=defensas_stats.toPandas()
defensas_stats.show()

Aquí se aprecia claramente que los defensas de la Bundesliga y la Ligue 1 son los mejores.

In [ ]:
# Configuración de la figura
plt.figure(figsize=(10, 8))

# Crear un scatter plot
for comp in defensas_stats_pd['Comp'].unique():
    subset = defensas_stats_pd[defensas_stats_pd['Comp'] == comp]
    plt.scatter(subset['avg_PressSucc%'], subset['avg_Interceptions/90'], label=comp, s=50)  # s controla el tamaño del punto

# Añadir leyenda
plt.legend(title='Competition')

# Añadir títulos y etiquetas
plt.title('Comparison of Average Defensive Performance Metrics by Competition')
plt.xlabel('Average Press Success %')
plt.ylabel('Average Interceptions per 90')

# Mostrar la gráfica
plt.show()

### Ejercicio 2:

Ahora hagamos algo un poco más interesante. Escoge algún jugador de todos los disponibles y toma 5 metricas, las que quieras. Debes concluir en qué percentil se encuentra el jugador en esas métricas que escogiste. Obviamente, vas a comparar sus valores con todos los demás con los que comparte posición y liga, para no tener un sesgo y que la información no pierda robustez. Por último, genera un DataFrame de Pyspark con todos los datos solicitados. Muestra el DataFrame y conviertelo a otro de tipo pandas. Muestra los dos.

In [ ]:
#Checo si existe el jugador que quiero
df_futbol.filter(df_futbol['Player'] == "Cristiano Ronaldo").show()

Observemos que es un delantero de la Serie A

In [ ]:
#Filtramos para sólo tener jugadores de su posición que jueguen en la misma liga
df_delanterosSerieA=df_futbol.filter((df_futbol['Comp']== 'it Serie A') & (col('pos').like("%FW%")))
df_delanterosSerieA.show()

In [ ]:
df_delanterosSerieA.count()

In [ ]:
#Creamos ventanas ordenadas con las métricas por las que vamos a evaluar
window_npG=Window.orderBy(col('npG/90').desc())
window_npGA=Window.orderBy(col('npG+A/90').desc())
window_passComp=Window.orderBy(col('PassCmp/90').desc())
window_keyPass=Window.orderBy(col('KeyPass/90').desc())
window_SuccDrib=Window.orderBy(col('SuccDrib/90').desc())

df_delanterosSerieA = df_delanterosSerieA.withColumn('percentile_npG/90', percent_rank().over(window_npG))
df_delanterosSerieA = df_delanterosSerieA.withColumn('percentile_npG+A/90', percent_rank().over(window_npGA))
df_delanterosSerieA = df_delanterosSerieA.withColumn('percentile_PassCmp/90', percent_rank().over(window_passComp))
df_delanterosSerieA = df_delanterosSerieA.withColumn('percentile_KeyPass/90', percent_rank().over(window_keyPass))
df_delanterosSerieA = df_delanterosSerieA.withColumn('percentile_SuccDrib/90', percent_rank().over(window_SuccDrib))

In [ ]:
cristiano= df_delanterosSerieA.filter(df_delanterosSerieA['Player'] == 'Cristiano Ronaldo')
cristiano_stats=cristiano.select('Player','percentile_npG/90', 'percentile_npG+A/90', 'percentile_PassCmp/90', 'percentile_KeyPass/90', 'percentile_SuccDrib/90')
cristiano_stats_pd=cristiano_stats.toPandas()

In [ ]:
cristiano_stats.show()

In [ ]:
cristiano_stats_pd